# Import

In [1]:
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Chargement de la donnée

In [2]:
class FER2013Dataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.data = []
        for label in os.listdir(directory):
            for image_file in os.listdir(os.path.join(directory, label)):
                self.data.append([os.path.join(directory, label, image_file), label])
                
        self.label_to_idx = {label: idx for idx, label in enumerate(os.listdir(directory))}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('L') 
        label = self.label_to_idx[label]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# On définit les transformations des images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# On crée les datasets
train_dataset = FER2013Dataset(directory='archive/train', transform=transform)
test_dataset = FER2013Dataset(directory='archive/test', transform=transform)

# On crée les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Le modèle

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Deep_Emotion(nn.Module):
    def __init__(self):
        '''
        Deep_Emotion class contains the network architecture.
        '''
        super(Deep_Emotion,self).__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,10,3)
        self.pool2 = nn.MaxPool2d(2,2)

        self.conv3 = nn.Conv2d(10,10,3)
        self.conv4 = nn.Conv2d(10,10,3)
        self.pool4 = nn.MaxPool2d(2,2)

        self.norm = nn.BatchNorm2d(10)

        self.fc1 = nn.Linear(810,50)
        self.fc2 = nn.Linear(50,7)

        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )

        self.fc_loc = nn.Sequential(
            nn.Linear(640, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 640)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)

        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self,input):
        out = self.stn(input)

        out = F.relu(self.conv1(out))
        out = self.conv2(out)
        out = F.relu(self.pool2(out))

        out = F.relu(self.conv3(out))
        out = self.norm(self.conv4(out))
        out = F.relu(self.pool4(out))

        out = F.dropout(out)
        out = out.view(-1, 810)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

# Fonction d'entrainement

In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using mps device


In [5]:
def train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=25):
    
    model.to(device)
    print("===================================Start Training===================================")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        validation_loss = 0.0
        correct = 0
        val_correct = 0
        total = 0
        val_total = 0
        
        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            val_outputs = model(inputs)
            val_loss = criterion(val_outputs, labels)  
            validation_loss += val_loss.item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (val_predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total

        val_epoch_loss = validation_loss / len(test_loader)
        val_epoch_acc = 100 * val_correct / val_total
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')
        print(f'Validation Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2f}%')

    print("===================================Training Finished===================================")



In [6]:
model = Deep_Emotion().to(device)

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
#optimizer = optim.Adam(model.parameters(), lr=0.005)

In [9]:
num_epochs = 1
#train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=num_epochs)

# Load and visualize model

In [11]:
import torch

model.load_state_dict(torch.load('model1.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
import argparse
import os
import cv2
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [15]:
transformation = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
classes = ('Angry', 'Disgust', 'Fear', 'Happy','Sad', 'Surprise', 'Neutral')


def load_img(path):
    img = Image.open(path)
    img = transformation(img).float()
    img = torch.autograd.Variable(img,requires_grad = True)
    img = img.unsqueeze(0)
    return img.to(device)

In [23]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms

# Charger le classificateur de visages pré-entraîné
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialiser la webcam
cap = cv2.VideoCapture(0)

# Transformation pour l'entrée du modèle
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((48, 48)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

while True:
    # Lire l'image de la webcam
    ret, img = cap.read()
    if not ret:
        break  # Sortir de la boucle si la capture a échoué

    # Convertir l'image en niveaux de gris pour la détection du visage
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Extraire la région d'intérêt (le visage)
        roi_gray = gray[y:y+h, x:x+w]

        # Transformer l'image pour l'adapter au modèle
        roi_gray_resized = cv2.resize(roi_gray, (48, 48))  # Redimensionner l'image pour correspondre à l'entrée du modèle
        roi_gray_resized = np.expand_dims(roi_gray_resized, axis=2)  # Ajouter une dimension de canal
        roi = transform(roi_gray_resized)  # Appliquer les transformations
        roi = roi.unsqueeze(0)  # Ajouter une dimension de batch
        roi = roi.to(device)  # Envoyer l'entrée à l'appareil approprié

        # Faire la prédiction
        with torch.no_grad():
            out = model(roi)
            _, predicted = torch.max(out.data, 1)
            prediction = predicted.item()  # Ici, vous devez mapper cet indice à l'étiquette réelle

        # Afficher le rectangle et l'étiquette autour du visage
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(prediction), (x, y-10), font, 0.9, (36,255,12), 2)

    # Afficher l'image résultante
    cv2.imshow('Emotion Recognition', img)

    # Quitter avec la touche 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()


/Users/nanditraore/anaconda3/lib/python3.10/site-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/Users/nanditraore/anaconda3/lib/python3.10/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import cv2
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
rectangle_bgr = (255, 255, 255)
img = np.zeros((500, 500))
text = "Some text in a box"
(text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
text_offset_x = 10
text_offset_y = img.shape[0] - 25

In [22]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
while True:

        # Read the frame
        _, img = cap.read()
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect the faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            roi = img[y:y+h, x:x+w]
            roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(roi,(48,48))
            cv2.imwrite("roi.jpg", roi)
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

        imgg = load_img("roi.jpg")
        out = model(imgg)
        pred = F.softmax(out)
        classs = torch.argmax(pred,1)
        wrong = torch.where(classs != 3,torch.tensor([1.]).cuda(),torch.tensor([0.]).cuda())
        classs = torch.argmax(pred,1)
        prediction = classes[classs.item()]
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (50, 50)
        fontScale = 1
        color = (255, 0, 0)
        thickness = 2
        img = cv2.putText(img, prediction, org, font,
                       fontScale, color, thickness, cv2.LINE_AA)

        cv2.imshow('img', img)
        # Stop if (Q) key is pressed
        k = cv2.waitKey(30)
        if k==ord("q"):
            break
cap.release()

FileNotFoundError: [Errno 2] No such file or directory: 'roi.jpg'